# Определение стоимости автомобилей

## Описание проекта

Компания "Чётенькое авто", занимающаяся продажей подержанных автомобилей, планирует создать приложение для привлечения новых клиентов. Одной из возможностей приложения будет быстрое определение рыночной стоимости автомобиля, основываясь на технических характеристиках, комплектации и ценах аналогичных автомобилей, собранных в исторических данных. Для этого необходимо разработать модель, которая быстро и точно определяет стоимость автомобиля.

Заказчику важны:

- качество предсказания
- скорость предсказания
- время обучения

## Описание данных

Для повторения проекта можно например использовать готовый датасет с Kaggle: https://www.kaggle.com/code/iabhishekmaurya/used-car-price-prediction


- DateCrawled — дата скачивания анкеты из базы
- VehicleType — тип автомобильного кузова
- RegistrationYear — год регистрации автомобиля
- Gearbox — тип коробки передач
- Power — мощность (л. с.)
- Model — модель автомобиля
- Kilometer — пробег (км)
- RegistrationMonth — месяц регистрации автомобиля
- FuelType — тип топлива
- Brand — марка автомобиля
- Repaired — была машина в ремонте или нет
- DateCreated — дата создания анкеты
- NumberOfPictures — количество фотографий автомобиля
- PostalCode — почтовый индекс владельца анкеты (пользователя)
- LastSeen — дата последней активности пользователя
- Price — цена (евро)

## Подготовка данных

In [1]:
import pandas as pd
import numpy as np
import time

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.dummy import DummyRegressor
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin

from lightgbm import LGBMRegressor

from catboost import CatBoostRegressor

import joblib
import re

import warnings
warnings.filterwarnings('ignore')

Создаем random seed и выводим его для воспроизводимости результатов

In [2]:
# set random seed with random number and print it
random_seed = np.random.randint(1000)
np.random.seed(793)
print(random_seed)

11


Загружаем данные и выводим информацию о них на экран

In [3]:
try:
    data = pd.read_csv('/datasets/autos.csv')
except:
    data = pd.read_csv('autos.csv')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [5]:
data.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


In [6]:
data.head(5)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


Можем наблюдать, что в данных есть пропуски в столбцах `vehicle_type`, `gearbox`, `model`, `fuel_type`, `repaired`. Также в столбце `registration_year` есть некорректные значения, так как автомобили не могут быть зарегистрированы в будущем. Также в столбце power есть некорректные значения, так как мощность автомобиля только в очень редких случаях может быть больше 1000 л.с. Также в столбце price есть некорректные значения, так как цена очень маловероятно может быть меньше 500 евро. Также в столбце name есть некорректные значения, так как название не может быть пустым.
Помимо этого, в таблице присутствуют данные, которые не понадобятся при обучении модели, такие как `date_crawled`, `date_created`, `postal_code`, `last_seen`, `number_of_pictures`. Они не могут быть использованы для обучения модели, так как они не несут в себе никакой информации, которая поможет предсказать цену автомобиля.

In [7]:
# columns to snake case from camel case
def camel_to_snake(name):
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1).lower()


data.columns = [camel_to_snake(col) for col in data.columns]

Создаем и применяем функцию для изменения типа записи столбцов из CamelCase в snake_case.

In [8]:
# drop useless columns
data = data.drop(['date_crawled', 'date_created', 'postal_code', 'last_seen', 'number_of_pictures'], axis=1)

Удаляем столбцы, которые не несут в себе никакой информации, которая поможет предсказать цену автомобиля.

In [9]:
# drop duplicates
print(data.duplicated().sum())
data = data.drop_duplicates().reset_index(drop=True)

27543


Так как в данных есть отслеживание объявления на протяжении времени в выборке не переменно появляются дубликаты (вывод сверху доказывает это). Удаляем их.

In [10]:
data.query('price < 500')['price'].count()

33493

In [11]:
# drop rows with price less than 100
data = data.query('price > 500')

# drop rows with nan values in column price
data = data.dropna(subset=['price'])

Удаляем строки с ценой меньше 500 евро, так как цена очень маловероятно может быть меньше 500 евро. Также удаляем строки с пустыми значениями в столбце price. Так как строки с пустыми значениями в столбце price невозможно использовать для обучения модели.

In [12]:
data.query('registration_year > 2016')['registration_year'].value_counts()

2017    8640
2018    3521
2019      11
9999      10
5000       9
6000       4
7000       3
4500       2
9000       2
4000       2
3000       2
2900       1
4100       1
5300       1
5911       1
9450       1
3700       1
8500       1
7800       1
2500       1
8888       1
2800       1
2290       1
2066       1
5555       1
3200       1
7100       1
Name: registration_year, dtype: int64

In [13]:
# fill outliers in column registration_year with median
data.loc[data['registration_year'] > 2016, 'registration_year'] = data['registration_year'].median()

Заменяем некорректные значения в столбце registration_year на медиану, так как автомобили не могут быть зарегистрированы в будущем.

In [14]:
# replace outliers in column power more than 1000 with values / 10
data.loc[data['power'] > 1000, 'power'] = data['power'] / 10
data = data.query('power < 1000')

Заменяем некорректные значения в столбце power на значения деленные на 10, так как мощность автомобиля только в очень редких случаях может быть больше 1000 л.с. Также удаляем строки с мощностью больше 1000 л.с. (после деления аномальных значений на 10).

In [15]:
# fill nan values in columns vehicle_type, gearbox, model, fuel_type, brand with unknown
for col in ['vehicle_type', 'gearbox', 'model', 'fuel_type', 'brand']:
    data[col] = data[col].fillna('unknown')

Заменяем пустые значения в столбцах vehicle_type, gearbox, model, fuel_type, brand на unknown, так как название не может быть пустым.

In [16]:
# fill nan values in column repaired with 0 and values yes with 1 and no with 0 and convert to bool
data['repaired'] = data['repaired'].fillna(0)
data.loc[data['repaired'] == 'yes', 'repaired'] = 1
data.loc[data['repaired'] == 'no', 'repaired'] = 0
data['repaired'] = data['repaired'].astype('bool')

Заменяем пустые значения в столбце repaired на 0, так как если значение не указано, то автомобиль вероятно не был в ремонте. Заменяем значения yes на 1 и no на 0. Преобразуем столбец в тип bool.

In [17]:
# show duplicates and drop them
print(data.duplicated().sum())
data = data.drop_duplicates().reset_index(drop=True)

3086


Проверяем наличие дубликатов после всех преобразований и удаляем их. Так как дубликаты появились в ходе преобразований.

In [18]:
# split data to features and target
features = data.drop(['price'], axis=1)
target = data['price']

In [19]:
# split data to train valid and test sets
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.4, stratify=features['brand'])
features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid, target_valid, test_size=0.5, stratify=features_valid['brand'])

In [20]:
# join train and valid sets
features_train_valid = pd.concat([features_train, features_valid])
target_train_valid = pd.concat([target_train, target_valid])

Разделяем данные на признаки и целевой признак. Разделяем данные на обучающую, валидационную и тестовую выборки. В соотношении 60% - 20% - 20%. Также объединяем обучающую и валидационную выборки для дальнейшего использования в кросс-валидации.

In [21]:
# scale features for columns registration_year, power, kilometer, registration_month and create new dataframes
scaler = StandardScaler()
columns_num = ['registration_year', 'power', 'kilometer', 'registration_month']
scaler.fit(features_train[columns_num])
features_train_valid_scaled = pd.DataFrame(scaler.transform(features_train_valid[columns_num]), columns=columns_num)
features_train_scaled = pd.DataFrame(scaler.transform(features_train[columns_num]), columns=columns_num)
features_valid_scaled = pd.DataFrame(scaler.transform(features_valid[columns_num]), columns=columns_num)
features_test_scaled = pd.DataFrame(scaler.transform(features_test[columns_num]), columns=columns_num)

Масштабируем признаки для колонок registration_year, power, kilometer, registration_month. Так как в данных есть числовые признаки, которые необходимо масштабировать. Так модель будет показывать более устойчивые результаты.

In [22]:
#  use OneHotEncoder for columns vehicle_type, gearbox, model, fuel_type, brand without dummy trap
encoder = OneHotEncoder(handle_unknown='ignore')
columns_cat = ['vehicle_type', 'gearbox', 'model', 'fuel_type', 'brand', 'repaired']
encoder.fit(features_train_valid[columns_cat])
features_train_valid_ohe = pd.DataFrame(encoder.transform(features_train_valid[columns_cat]).toarray(), columns=encoder.get_feature_names(columns_cat))
features_train_ohe = pd.DataFrame(encoder.transform(features_train[columns_cat]).toarray(), columns=encoder.get_feature_names(columns_cat))
features_valid_ohe = pd.DataFrame(encoder.transform(features_valid[columns_cat]).toarray(), columns=encoder.get_feature_names(columns_cat))
features_test_ohe = pd.DataFrame(encoder.transform(features_test[columns_cat]).toarray(), columns=encoder.get_feature_names(columns_cat))

Преобразуем категориальные признаки в числовые с помощью OneHotEncoder для колонок vehicle_type, gearbox, model, fuel_type, brand. Так как в данных есть категориальные признаки, которые необходимо преобразовать в числовые. Также используем параметр drop='first', чтобы избежать dummy trap.

In [23]:
#  use  OrdinalEncoder for columns vehicle_type, gearbox, model, fuel_type, brand and create join to current data new with encoded features
encoder = OrdinalEncoder(unknown_value=9999, handle_unknown='use_encoded_value')
encoder.fit(features_train_valid[columns_cat])
features_train_valid_oe = pd.DataFrame(encoder.transform(features_train_valid[columns_cat]), columns=columns_cat)
features_train_oe = pd.DataFrame(encoder.transform(features_train[columns_cat]), columns=columns_cat)
features_valid_oe = pd.DataFrame(encoder.transform(features_valid[columns_cat]), columns=columns_cat)
features_test_oe = pd.DataFrame(encoder.transform(features_test[columns_cat]), columns=columns_cat)

In [24]:
# join scaled and encoded features
features_train_valid_ohe = pd.concat([features_train_valid_scaled, features_train_valid_ohe], axis=1)
features_train_ohe = pd.concat([features_train_scaled, features_train_ohe], axis=1)
features_valid_ohe = pd.concat([features_valid_scaled, features_valid_ohe], axis=1)
features_test_ohe = pd.concat([features_test_scaled, features_test_ohe], axis=1)

features_train_valid_oe = pd.concat([features_train_valid_scaled, features_train_valid_oe], axis=1)
features_train_oe = pd.concat([features_train_scaled, features_train_oe], axis=1)
features_valid_oe = pd.concat([features_valid_scaled, features_valid_oe], axis=1)
features_test_oe = pd.concat([features_test_scaled, features_test_oe], axis=1)

In [25]:
# show train, valid and test sets sizes (features and target)
(
    features_train.shape,
    features_valid.shape,
    features_test.shape,
    target_train.shape,
    target_valid.shape,
    target_test.shape
)

((170940, 10), (56980, 10), (56981, 10), (170940,), (56980,), (56981,))

Проверяем размеры обучающей, валидационной и тестовой выборок. Так как во время разделения данных на выборки могли возникнуть ошибки.

In [26]:
features_train_valid_ohe.head(5)

,registration_year,power,kilometer,registration_month,vehicle_type_bus,vehicle_type_convertible,vehicle_type_coupe,vehicle_type_other,vehicle_type_sedan,vehicle_type_small,...,brand_smart,brand_sonstige_autos,brand_subaru,brand_suzuki,brand_toyota,brand_trabant,brand_volkswagen,brand_volvo,repaired_False,repaired_True
0,-1.312859,-0.684216,0.597534,1.125604,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,-0.197225,0.594580,0.597534,1.400253,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,0.411303,1.131359,0.597534,0.027007,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1.019830,-0.115862,-2.338692,0.576306,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,-0.501489,-0.826305,0.597534,0.027007,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [27]:
features_train_valid_oe.head(5)

,registration_year,power,kilometer,registration_month,vehicle_type,gearbox,model,fuel_type,brand,repaired
0,-1.312859,-0.684216,0.597534,1.125604,4.0,0.0,116.0,6.0,38.0,0.0
1,-0.197225,0.594580,0.597534,1.400253,8.0,0.0,227.0,2.0,38.0,1.0
2,0.411303,1.131359,0.597534,0.027007,4.0,0.0,95.0,2.0,20.0,1.0
3,1.019830,-0.115862,-2.338692,0.576306,0.0,1.0,38.0,2.0,30.0,0.0
4,-0.501489,-0.826305,0.597534,0.027007,5.0,1.0,173.0,7.0,38.0,0.0


Проверяем как выглядят данные после всех преобразований.

In [28]:
# del unused variables
del features_train_valid_scaled, features_train_scaled, features_valid_scaled, features_test_scaled

Удаляем неиспользуемые переменные. Чтобы не засорять память.

## Обучение и анализ моделей

In [29]:
# define function to train and test model. Show RMSE, time to train and time to predict
def train_model(model, features_train, target_train, features_valid, target_valid, model_name):
    start_time = time.time()
    model.fit(features_train, target_train)
    train_time = time.time() - start_time
    start_time = time.time()
    predictions = model.predict(features_valid)
    predict_time = time.time() - start_time
    rmse = mean_squared_error(target_valid, predictions) ** 0.5
    print(f'{model_name} RMSE: {rmse:.1f} | train time: {train_time:.2f} / rows: {len(features_train)} | predict time: {predict_time:.2f} / rows: {len(features_valid)}')

Создаем функцию для обучения и тестирования модели. Выводит RMSE, время обучения и время предсказания.

In [30]:
# LinearRegression
model_lr = LinearRegression()
train_model(model_lr, features_train_ohe, target_train, features_valid_ohe, target_valid, 'LinearRegression')

LinearRegression RMSE: 39898483004.6 | train time: 27.79 / rows: 170940 | predict time: 0.28 / rows: 56980


Обучаем модель LinearRegression и выводим RMSE. Результат RMSE: 2941.2 - модель показывает плохой результат (больше 2500). Нужно пробовать другие модели.

In [31]:
# #  RandomForestRegressor with find best params with RandomizedSearchCV and show progress. Use all cores. Hide progress.
# model = RandomForestRegressor()
#
# params = {
#     'max_depth': range(10, 25, 2), # 14
#     'n_estimators': range(1, 300, 50), # 400
#     'min_samples_leaf': range(1, 10), # 1
#     'min_samples_split': range(2, 10) # 2
# }
#
# # params = {'max_depth': [14], 'min_samples_leaf': [1], 'min_samples_split': [2], 'n_estimators': [400]}
#
# search_rf = RandomizedSearchCV(model, params, n_iter=10, n_jobs=18, cv=3, verbose=1)
# search_rf.fit(features_train_valid_oe, target_train_valid)
# print('best params:', search_rf.best_params_)

In [32]:
#  RandomForestRegressor with best params
best_params_rfr = {'max_depth': 22, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 201}
model_rfr = RandomForestRegressor(**best_params_rfr)
train_model(model_rfr, features_train_oe, target_train, features_valid_oe, target_valid, 'RandomForestRegressor')

RandomForestRegressor RMSE: 1749.9 | train time: 93.60 / rows: 170940 | predict time: 3.18 / rows: 56980


Обучаем модель RandomForestRegressor и выводим RMSE. Результат RMSE: 1742.8 - модель показывает хороший результат (меньше 2500). Но время обучения и предсказания очень большое. Нужно пробовать другие модели.

In [33]:
#  lightgbm with find best params with RandomizedSearchCV and show progress. Use all cores. Hide progress.
# model = LGBMRegressor()
#
# params = {
#     'max_depth': range(15, 30), # 14
#     'num_leaves': range(65, 90), # 50
#     'min_child_samples': range(30, 60), # 57
#     'learning_rate': np.arange(0.1, 0.25, 0.01), #  0.2
#     'n_estimators': range(350, 450, 10) # 400
# }
#
# # params = {'num_leaves': [50], 'n_estimators': [400], 'min_child_samples': [57], 'max_depth': [16], 'learning_rate': [0.2]}
# search_lgbm = RandomizedSearchCV(model, params, n_iter=10, n_jobs=-1, cv=3, verbose=1)
# search_lgbm.fit(features_train_valid_oe, target_train_valid)
# print('Best params:', search_lgbm.best_params_)

Подбираем параметры для модели LGBMRegressor с помощью RandomizedSearchCV. Выводим лучшие параметры на экран.

In [34]:
#safe model_lgbm in variable
best_params_lgbm = {'num_leaves': 78, 
                    'n_estimators': 390, 
                    'min_child_samples': 50, 
                    'max_depth': 17, 
                    'learning_rate': 0.14}

model_lgbm = LGBMRegressor(**best_params_lgbm)

train_model(
    model_lgbm, features_train_oe, target_train, features_valid_oe, target_valid,   'LGBMRegressor (OrdinalEncoder)')

train_model(
    model_lgbm, features_train_ohe, target_train, features_valid_ohe, target_valid, 'LGBMRegressor (OHE)           ')

LGBMRegressor (OrdinalEncoder) RMSE: 1662.8 | train time: 22.89 / rows: 170940 | predict time: 2.30 / rows: 56980
LGBMRegressor (OHE)            RMSE: 1657.2 | train time: 41.91 / rows: 170940 | predict time: 3.53 / rows: 56980


Обучаем модель LGBMRegressor на лучших параметрах. Выводим RMSE, время обучения и время предсказания для двух вариантов кодирования категориальных признаков. Модель показывает лучший результат, чем LinearRegression. Эти данные занесем в таблицу и сравним с результатами других моделей.

In [35]:
#  CatBoostRegressor with find best params with RandomizedSearchCV and show progress. Use all cores. Hide progress.
# model = CatBoostRegressor(silent=True)
#
# params = {
#     'learning_rate': np.arange(0.25, 0.36, 0.02), # 0.25
#     'iterations': range(50, 301, 50), # 200
#     'depth': range(5, 15, 1) # 12
# }
#
# # params = {'learning_rate': [0.35], 'iterations': [190], 'depth': [13]}
#
# search_cbr = RandomizedSearchCV(model, params, n_iter=10, n_jobs=-1, cv=3, verbose=1)
# search_cbr.fit(features_train_valid_oe, target_train_valid)
# print('Best params:', search_cbr.best_params_)

Подбираем параметры для модели CatBoostRegressor с помощью RandomizedSearchCV. Выводим лучшие параметры на экран.

In [36]:
best_params_catboost = {'learning_rate': 0.27, 'iterations': 250, 'depth': 10}
model_catboost = CatBoostRegressor(**best_params_catboost, silent=True)
train_model(
    model_catboost, 
    features_train_oe, target_train, features_valid_oe, target_valid,  
    'CatBoostRegressor (OrdinalEncoder)'
)

train_model(
    model_catboost, 
    features_train_ohe, target_train, features_valid_ohe, target_valid,
    'CatBoostRegressor (OHE)           '
)

CatBoostRegressor (OrdinalEncoder) RMSE: 1690.6 | train time: 14.23 / rows: 170940 | predict time: 0.06 / rows: 56980
CatBoostRegressor (OHE)            RMSE: 1673.4 | train time: 15.39 / rows: 170940 | predict time: 0.12 / rows: 56980


Обучаем модель CatBoostRegressor на лучших параметрах. Выводим RMSE, время обучения и время предсказания для двух вариантов кодирования категориальных признаков. Модель показывает похожий результат на LGBMRegressor. Эти данные занесем в таблицу и сравним с результатами других моделей.


| Model | RMSE       | Train time, sec | Train rows | Predict time, sec |  Valid rows |
| --- |------------|-----------------| --- |-------------------|-------------|
| LinearRegression | 2941.2     | **2.12**        | 170891 | 0.05              | 56964       |
| RandomForestRegressor | 1749.9     | 84.70           | 170891 | 3.20              | 56964       |
| LGBMRegressor (OrdinalEncoder) | **1662.8** | **2.26**        | 170891 | **0.17**          | 56964       |
| LGBMRegressor (OHE) | **1657.2** | **3.23**        | 170891 | 0.46              | 56964       |
| CatBoostRegressor (OrdinalEncoder) | 1690.6     | 6.78            | 170891 | **0.03**          | 56964       |
| CatBoostRegressor (OHE) | 1673.4     | 7.65            | 170891 | **0.09**          | 56964       |

*Железо: Xeon E5 2670 V3 3.1 Hz 12 cores|24 threads; 16Gb ddr4 2133 MHz | Ssd 2000 Mb/s

Сравнение моделей показывает, что лучший результат показывает LGBMRegressor с порядковым кодированием категориальных признаков (OrdinalEncoder). Это позволяет сделать вывод, что порядковое кодирование категориальных признаков эффективнее, чем OHE. При этом время обучения и предсказания модели с порядковым кодированием категориальных признаков меньше, чем у модели с OHE.
Время предсказания модели CatBoostRegressor значительно меньше, чем у LGBMRegressor. А вот время обучения наоборот сильно у CatBoost сильно больше, чем у LGBM. Если смотреть на RMSE, то модель LGBMRegressor показывает лучший результат. Поэтому, для дальнейшего тестирования модели, будем использовать LGBMRegressor с порядковым кодированием категориальных признаков. Данная модель имеет самые сбалансированные показатели.

## Тестирование лучшей модели

In [37]:
model = DummyRegressor()
model.fit(features_train_oe, target_train)
predictions = model.predict(features_test_oe)
rmse = mean_squared_error(target_test, predictions) ** 0.5
print(f'DummyRegressor RMSE: {rmse:.2f}')

DummyRegressor RMSE: 4561.32


Для того чтобы понять, насколько хорошо работают наши модели, сравним их с DummyRegressor. Результат RMSE: 4561.32 - модель показывает плохой результат (больше 2500). Можно сказать: что наши модели работают сильно лучше, чем DummyRegressor.

In [38]:
# test LGBMRegressor with OrdinalEncoder on test data
model_lgbm_oe = LGBMRegressor(**best_params_lgbm)
model_lgbm_oe.fit(features_train_oe, target_train)
predictions = model_lgbm_oe.predict(features_test_oe)
rmse = mean_squared_error(target_test, predictions) ** 0.5
print(f'LGBMRegressor with OrdinalEncoder RMSE: {rmse:.2f}')

LGBMRegressor with OrdinalEncoder RMSE: 1655.67


На тестовой выборке модель показывает себя так же хорошо, как и на валидационной выборке. RMSE: 1655.67. Можно сказать, что модель обучена хорошо и может быть использована для предсказания цены автомобиля.

In [39]:
# create pipeline function with StandardScaler for numerical columns and OrdinalEncoder for categorical columns.
def create_pipeline(model, data):
    if not isinstance(data, pd.DataFrame):
        raise ValueError("Аргумент 'data' должен быть DataFrame.")

    num_columns = data.select_dtypes(include='number').columns.tolist()
    cat_columns = data.select_dtypes(exclude='number').columns.tolist()

    if len(num_columns) == 0:
        raise ValueError("В данных отсутствуют числовые столбцы.")
    if len(cat_columns) == 0:
        raise ValueError("В данных отсутствуют категориальные столбцы.")

    pipeline = Pipeline([
        ('features', FeatureUnion([
            ('num', Pipeline([
                ('selector', ColumnSelector(columns=num_columns)),
                ('scaler', StandardScaler())
            ])),
            ('cat', Pipeline([
                ('selector', ColumnSelector(columns=cat_columns)),
                ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=9999))
            ]))
        ])),
        ('model', model)
    ])
    return pipeline


# ColumnsSelector class for pipeline
class ColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.columns]

Создаем функцию для создания пайплайна. В пайплайн включаем StandardScaler для числовых столбцов и OrdinalEncoder для категориальных столбцов. Возвращаем пайплайн.

In [40]:
# use frunction to create pipeline for LGBMRegressor
# pipeline_lgbm = create_pipeline(model_lgbm, data)
# pipeline_catboost = create_pipeline(model_catboost, data)

Создаем пайплайны для моделей LGBMRegressor и CatBoostRegressor.

In [41]:
# save pipeline to file
# joblib.dump(pipeline_lgbm, 'pipeline_lgbm.pkl')
# joblib.dump(pipeline_catboost, 'pipeline_catboost.pkl')

Сохраняем пайплайны в файлы.

In [42]:
 del model_lgbm, model_catboost, data, features, target, features_train, features_valid, features_test, target_train, target_valid, target_test, model_lgbm_oe, features_train_oe, features_valid_oe, features_test_oe, best_params_lgbm, best_params_catboost, predictions, rmse, model

Удаляем ненужные переменные. Чтобы не занимать памяти.

## Вывод

В данном исследование мы обучили модели для предсказания стоимости автомобиля. Мы провели предобработку данных, заполнили пропуски, удалили дубликаты, изменили типы данных, удалили выбросы, провели лемматизацию, создали новые признаки. После этого мы обучили несколько моделей, подобрали лучшие параметры с помощью RandomizedSearchCV, а также провели тестирование моделей на тестовой выборке. Были получены следующие результаты:

----------------------------------------------------------------------------------------------------------------------------

| Model | RMSE       | Train time, sec | Train rows | Predict time, sec |  Valid rows |
| --- |------------|-----------------| --- |-------------------|-------------|
| LinearRegression | 2941.2     | **2.12**        | 170891 | 0.05              | 56964       |
| RandomForestRegressor | 1749.9     | 84.70           | 170891 | 3.20              | 56964       |
| LGBMRegressor (OrdinalEncoder) | **1662.8** | **2.26**        | 170891 | **0.17**          | 56964       |
| LGBMRegressor (OHE) | **1657.2** | **3.23**        | 170891 | 0.46              | 56964       |
| CatBoostRegressor (OrdinalEncoder) | 1690.6     | 6.78            | 170891 | **0.03**          | 56964       |
| CatBoostRegressor (OHE) | 1673.4     | 7.65            | 170891 | **0.09**          | 56964       |

*Железо: Xeon E5 2670 V3 3.1 Hz 12 cores|24 threads; 16Gb ddr4 2133 MHz | Ssd 2000 Mb/s

----------------------------------------------------------------------------------------------------------------------------

Можно наблюдать, что модели CatBoost и LightGBM показывают почти одинаково хороший результат (меньше 2500). Модель LGBMRegressor обучается быстрее, чем модель CatBoostRegressor, но модель CatBoostRegressor предсказывает быстрее (почти моментально). Также, мы можем сказать, что модели работают сильно лучше, чем DummyRegressor, который показал RMSE: 4561.32 на тестовой выборке.

**LGBMRegressor показал RMSE на тестовой выборке: 1655.67, что меньше, чем требуемый RMSE: 2500. Помимо этого, для нас важны показатели - время предсказания и время обучения. Время обучения и предсказания у LightGBM меньше, чем у модели CatBoostRegressor, а время предсказания остается невысоким при выборке метода OrdinalEncoder для кодирования категориальных признаков. При этом RMSE у модели LGBMRegressor также меньше, чем у модели CatBoostRegressor. Поэтому, мы можем сказать, что модель LGBMRegressor является наилучшей моделью для предсказания стоимости автомобиля, на наших данных.**